In [ ]:
import os
import time
from datetime import datetime, timedelta
import glob
from multiprocessing import Pool
import traceback

from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt
import zarr

from lmsiage.mesh_file import MeshFile

In [ ]:
lag_dir = 'nextsim/mesh'

start_file_path = None
if os.path.exists('start_file_path.txt'):
    with open('start_file_path.txt', 'rt') as f:
        start_file_path = f.readline().strip()

if not start_file_path:
    start_file_path = sorted(glob.glob(f'{lag_dir}/*/????0915.zip'))[0]
    with open('start_file_path.txt', 'w') as f:
        f.write(f'{start_file_path}\n')
start_file_date = datetime.strptime(os.path.basename(start_file_path).split('.')[0], '%Y%m%d')
print(start_file_path, start_file_date)

In [ ]:
for i in range(300):
    src_file_date = start_file_date + timedelta(i)
    dst_file_date = start_file_date + timedelta(i + 1)
    src_file_path = src_file_date.strftime(f'{lag_dir}/%Y/%Y%m%d.zip')
    dst_file_path = dst_file_date.strftime(f'{lag_dir}/%Y/%Y%m%d.zip')
    if not os.path.exists(dst_file_path):
        break
    print(src_file_path, dst_file_path)
    src_mf = MeshFile(src_file_path)
    dst_mf = MeshFile(dst_file_path)
    src_names = src_mf.read_names()
    myi_names = [n for n in src_names if n.startswith('sic') and len(n) > 3]
    src_myi = src_mf.load(myi_names)

    src2dst, w, ar, sic = dst_mf.load(['src2dst', 'w', 'ar', 'sic'], as_dict=False)
    ar[ar == 0] = 0.01
    dst_myi = {}
    for name, c0 in src_myi.items():
        c1 = np.zeros(src2dst[:,1].max()+1)
        np.add.at(c1, src2dst[:,1], c0[src2dst[:,0]] * w)
        c1 /= ar
        c1 = np.clip(c1, 0, 100)
        c1 = np.where(c1 > sic, sic, c1)
        dst_myi[name] = c1
    dst_mf.save(dst_myi, mode='a')
    with open('start_file_path.txt', 'w') as f:
        f.write(f'{dst_file_path}\n')